TO DO: header

Authors: Andreia Dourado, Julia Gschwend, Bruno Moraes

Notebook adapted from Melissa Graham example notebook \<details later\> and [discussion in Rubin Community Forum](https://community.lsst.org/t/adql-question-selecting-a-random-subset-of-objects-in-dp0/8022). 

rename notebook dp0.2 to dp02

This notebook is set up to be executed on the [Rubin Science Platform](https://data.lsst.cloud/)


#### Importing packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from lsst.rsp import get_tap_service, retrieve_query

In [ ]:
pd.set_option('display.max_rows', 20)
%matplotlib inline
service = get_tap_service("tap")

### Query 

https://dm.lsst.org/sdm_schemas/browser/dp02.html 

In [ ]:
pd.set_option('display.max_rows', 20)
%matplotlib inline
service = get_tap_service('tap')

In [ ]:
#query = ' '.join(['SELECT obj.objectId ', 
        #', ts.ra, ts.dec, ',
        #'i_cModelFlux, i_cModelFluxErr, scisql_nanojanskyToAbMag(i_cModelFlux) as mag_i, ',  
        #'scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) as magerr_i ',  
        #'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
        #'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
        #'ON mt.id_truth_type=ts.id_truth_type ',
        #'JOIN dp02_dc2_catalogs.Object AS obj ',
        #'ON mt.match_objectId=obj.objectId ', 
        #'WHERE obj.detect_isPrimary=1 AND scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) < 0.2',
        #'AND ts.truth_type=1 ',
        #'AND obj.detect_isPrimary=1'])

In [ ]:
esta query leva x tempo

In [ ]:
%%time
query = "SELECT objectId  FROM dp02_dc2_catalogs.Object WHERE detect_isPrimary=1 AND scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) < 0.2"
results = service.search(query)
df = service.search(query).to_table().to_pandas()
#df

Save the result to a file for general use later:


In [ ]:
df.to_csv('objectIds_all_galaxies_sn_max_5.csv')


In [ ]:
%%time
query = "SELECT objectId, scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) as magerr, detect_isPrimary FROM dp02_dc2_catalogs.Object limit 5"
results = service.search(query)
df = service.search(query).to_table().to_pandas()

In [ ]:
%%time
query = "SELECT objectId, scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) as magerr  FROM dp02_dc2_catalogs.Object WHERE detect_isPrimary=1"
results = service.search(query)
df = service.search(query).to_table().to_pandas()

In [ ]:
#df

Save the result to a file for general use later:


In [ ]:
df.to_csv('objectIds_all_galaxies.csv')


select $S/N > 5$

In [ ]:
# %%time
# query = '\n'.join(['SELECT mt.id_truth_type, mt.match_objectId, obj.objectId, ts.ra, ts.dec, ts.redshift,scisql_nanojanskyToAbMag(obj.u_cModelFlux) as mag_u, scisql_nanojanskyToAbMag(obj.g_cModelFlux) as mag_g, scisql_nanojanskyToAbMag(obj.r_cModelFlux) as mag_r, scisql_nanojanskyToAbMag(obj.i_cModelFlux) as mag_i, scisql_nanojanskyToAbMag(obj.z_cModelFlux) as mag_z,scisql_nanojanskyToAbMag(obj.y_cModelFlux) as mag_y, scisql_nanojanskyToAbMagSigma(u_calibFlux, u_calibFluxErr) as magerr_u, scisql_nanojanskyToAbMagSigma(g_calibFlux, g_calibFluxErr) as magerr_g, scisql_nanojanskyToAbMagSigma(r_calibFlux, r_calibFluxErr) as magerr_r, scisql_nanojanskyToAbMagSigma(i_calibFlux, i_calibFluxErr) as magerr_i, scisql_nanojanskyToAbMagSigma(z_calibFlux, z_calibFluxErr) as magerr_z, scisql_nanojanskyToAbMagSigma(y_calibFlux, y_calibFluxErr) as magerr_y ',
#         'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
#         'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
#         'ON mt.id_truth_type=ts.id_truth_type ',
#         'JOIN dp02_dc2_catalogs.Object AS obj ',
#         'ON mt.match_objectId=obj.objectId ', 
#         'WHERE obj.objectId IN'+obj_ids,
#         'AND ts.truth_type=1 ',
#         'AND obj.detect_isPrimary=1'])


##### 2.1 Selecting a random objectIds fraction

In [ ]:
rsp_user = 'gschwend'
#replace 
filename = f'/scratch/{rsp_user}/WORK/pz-compute/objectIds_all_galaxies_sn_max_5.csv'
random.seed(4321)
random_ids = pd.read_csv(filename, 
           skiprows=lambda x: x > 0 and random.random() >=0.006) 

In [ ]:
len(random_ids)

In [ ]:
random_ids.to_csv('random_ids1.csv')

Due to a limit of data that can be obtained on RSP, it was possible to obtain 50000 objects in each query. The following steps were repeated until to obtain all data. 

#### 2.2 Writting the objectIds in the query format

In [ ]:
ids = np.squeeze((random_ids.values)[50000:100000])
len(ids)

In [ ]:
obj_ids = str(tuple(ids))

#### 2.3 JOIN Object table and TruthSummary table

In [ ]:
%%time
query = '\n'.join(['SELECT mt.id_truth_type, mt.match_objectId, obj.objectId, ts.ra, ts.dec, ts.redshift,scisql_nanojanskyToAbMag(obj.u_cModelFlux) as mag_u, scisql_nanojanskyToAbMag(obj.g_cModelFlux) as mag_g, scisql_nanojanskyToAbMag(obj.r_cModelFlux) as mag_r, scisql_nanojanskyToAbMag(obj.i_cModelFlux) as mag_i, scisql_nanojanskyToAbMag(obj.z_cModelFlux) as mag_z,scisql_nanojanskyToAbMag(obj.y_cModelFlux) as mag_y, scisql_nanojanskyToAbMagSigma(u_calibFlux, u_calibFluxErr) as magerr_u, scisql_nanojanskyToAbMagSigma(g_calibFlux, g_calibFluxErr) as magerr_g, scisql_nanojanskyToAbMagSigma(r_calibFlux, r_calibFluxErr) as magerr_r, scisql_nanojanskyToAbMagSigma(i_calibFlux, i_calibFluxErr) as magerr_i, scisql_nanojanskyToAbMagSigma(z_calibFlux, z_calibFluxErr) as magerr_z, scisql_nanojanskyToAbMagSigma(y_calibFlux, y_calibFluxErr) as magerr_y ',
        'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
        'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
        'ON mt.id_truth_type=ts.id_truth_type ',
        'JOIN dp02_dc2_catalogs.Object AS obj ',
        'ON mt.match_objectId=obj.objectId ', 
        'WHERE obj.objectId IN'+obj_ids,
        'AND ts.truth_type=1 ',
        'AND obj.detect_isPrimary=1'])

#print(query)
#print(' ')
#results = service.search(query).to_table().to_pandas()#converte para tabela astropy e depois pra pandas dataframe
#print('len(results) = ', len(results))

Due to a time limit imposed on synchronous query, it was suggested the following asynchronous query:

In [ ]:
job = service.submit_job(query)
print('Job URL is', job.url)
print('Job phase is', job.phase)

This run take about 20 minutes.

In [ ]:
%%time
job.run()

In [ ]:
print('Job phase is now ', job.phase) 

In [ ]:
print(job.url)
retrieved_job = retrieve_query(job.url)
retrieved_results = retrieved_job.fetch_result().to_table().to_pandas()

In [ ]:
len(retrieved_results)

In [ ]:
async_results.to_csv("random_data_1.csv")

On-sky spatial distribuition

In [ ]:
plt.scatter(async_results['ra'],async_results['dec'], c=async_results['redshift'],cmap='turbo',s=1)
plt.colorbar()

#### 4. Final sample

Join all query results in only data frame:

In [ ]:
num = #quantidade de queries rodadas + 1 
filenames = ["DadosRandom" + str(i) + ".csv" for i in range(1, num)] 

In [ ]:
data = pd.DataFrame()
for filename in filenames:
    df = pd.read_csv(filename)
    pd.concat([data, df])

In [ ]:
data_specz=data[['objectId','ra','dec','redshift']]

On-sky spatial distribuition

In [ ]:
plt.scatter(data_specz['ra'],data_specz['dec'], c=data_specz['redshift'],cmap='turbo',s=1)
plt.colorbar()

In [ ]:
data_specz.to_csv("specz_data.csv")

In [ ]:
data.to_csv('random_data.csv') 